In [2]:
%matplotlib inline
import os
import pickle
import json
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats as stats
import dotenv
dotenv.load_dotenv()

True

In [3]:
plt.style.use('../style/style-formal.mplstyle')

In [4]:
RESEARCH_DATA = os.environ.get("RESEARCH_DATA")

EL = "attempt_1"
PIDS = ["Lang", "Mockito", "Math", "Time", "Chart"]

STD_METHOD = "lineCnt70_mutCnt7_tcsAll"
CMP_METHOD = "lineCnt70_mutCnt7_tcsReduced"

EXPERIMENT_DIRNAME = "experiment_raw_results"
REPEAT_CNT = 10
K_FOLD = 10

EXP_DIR = os.path.join(RESEARCH_DATA, f"{EL}/dlfl_out/{EXPERIMENT_DIRNAME}")
assert os.path.exists(EXP_DIR), f"Result directory does not exist: {EXP_DIR}"

In [5]:

def get_ranks(method_name):
    rid2bid2ranks = {}
    for rid in range(1, REPEAT_CNT+1):
        method_dir = os.path.join(EXP_DIR, f"repeat_{rid}/methods/{method_name}")
        assert os.path.exists(method_dir), f"Method directory does not exist: {method_dir}"
        rid2bid2ranks[rid] = {}
        for kid in range(1, K_FOLD+1):
            result_file = os.path.join(method_dir, f"results/result_{kid}.txt")
            assert os.path.exists(result_file), f"Result file does not exist: {result_file}"

            with open(result_file, 'r') as f:
                lines = f.readlines()
                line_id = 0
                while line_id < len(lines):
                    line = lines[line_id].strip()
                    if line.startswith("==="):
                        bid_name = line.strip().split("===")[-1].strip()
                        if bid_name not in rid2bid2ranks[rid]:
                            rid2bid2ranks[rid][bid_name] = []
                        line_id += 1
                        while line_id < len(lines):
                            new_line = lines[line_id].strip()
                            if new_line.startswith("==="):
                                line_id -= 1
                                break
                            rank = int(new_line.strip())
                            rid2bid2ranks[rid][bid_name].append(rank)
                            line_id += 1
                    line_id += 1
    return rid2bid2ranks


In [6]:
STD_BID2RANKS = get_ranks(STD_METHOD)
CMP_BID2RANKS = get_ranks(CMP_METHOD)

In [7]:
BID_LIST = list(STD_BID2RANKS[1].keys())
std_rank_list = []
cmp_rank_list = []
for bid in BID_LIST:
    for rid in range(1, REPEAT_CNT+1):
        std_rank_list.append(np.mean(STD_BID2RANKS[rid][bid]))
        cmp_rank_list.append(np.mean(CMP_BID2RANKS[rid][bid]))
print(f"STD LEN: {len(std_rank_list)}")
print(f"CMP LEN: {len(cmp_rank_list)}")

STD LEN: 2570
CMP LEN: 2570


In [8]:
wilcoxon_stat, p_value = stats.wilcoxon(
    x=std_rank_list,
    y=cmp_rank_list,
    zero_method='zsplit',
    alternative='two-sided'
)

In [9]:
print(f"wilcoxon_stat: {wilcoxon_stat}")
print(f"p_value: {p_value}")

wilcoxon_stat: 1607888.0
p_value: 0.24214174936083988
